# Fine-Tuning SwinV2 for Real vs. AI-Generated Image Classification

This notebook provides a step-by-step guide to training a **SwinV2 Small** model to classify images as either 'real' or 'fake' (AI-generated).

The process covers:
1.  **Setup**: Mounting Google Drive and installing necessary libraries.
2.  **Data Preparation**: Loading your `metadata.json` and splitting the dataset.
3.  **Dataset Creation**: Using a custom PyTorch `Dataset` for efficient data loading.
4.  **Model Configuration**: Loading the pre-trained SwinV2 model and adapting it for our binary classification task.
5.  **Training**: Fine-tuning the model using the Hugging Face `Trainer` API.
6.  **Inference**: Using the trained model to make predictions on new images.

### Step 1: Mount Google Drive & Install Libraries

First, we connect to Google Drive to access our dataset. Then, we install the required Python packages from Hugging Face and other utilities.

In [1]:

# Install required libraries quietly
!pip install -q transformers datasets evaluate accelerate pandas scikit-learn

### Step 2: Prepare the Data

Here, we'll load the `metadata.json` file into a pandas DataFrame. We'll construct the full path to each image and convert the 'real'/'fake' status into numerical labels (0/1). Finally, we split the data into training and validation sets.

In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# --- Configuration ---
# ⬇️ IMPORTANT: Change this path to where your 'data' folder is located in Google Drive
BASE_DIR = "./data"
METADATA_FILE = os.path.join(BASE_DIR, "metadata.json")

# --- Load and Process Metadata ---
print(f"Loading metadata from: {METADATA_FILE}")
df = pd.read_json(METADATA_FILE)

# The 'image_file' column might be a list, let's flatten it if so
# This handles cases where metadata.json has entries like [["real/image1.png"]]
if isinstance(df['image_file'].iloc[0], list):
    df['image_file'] = df['image_file'].str[0]

# Create the full, absolute path for each image file
df['full_path'] = df['image_file'].apply(lambda x: os.path.join(BASE_DIR, x))

# Create a numerical label: 0 for 'real', 1 for 'fake'
df['label'] = df['status'].apply(lambda x: 0 if x == 'real' else 1)

# --- Split the Data ---
# Split into training (80%) and validation (20%) sets
# stratify=df['label'] ensures both sets have a similar ratio of real/fake images
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Reset index for easier access later
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

print(f"\nTotal samples: {len(df)}")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print("\nSample of the training dataframe:")
print(train_df.head())

Loading metadata from: ./data\metadata.json

Total samples: 14062
Training samples: 11249
Validation samples: 2813

Sample of the training dataframe:
      id                  image_file  \
0  13103  real/hf_unsplash_41569.jpg   
1  12568  real/hf_unsplash_28579.jpg   
2  10431  real/hf_real_image_261.jpg   
3   6411  fake_DALLE/4676717700.webp   
4   2607  fake_SD/Images_sd3 432.jpg   

                                             prompts platform status  \
0                                               real     None   real   
1                                               real     None   real   
2                                               real     None   real   
3  A drummer is playing while wearing sunglasses,...  dall-E3   fake   
4   A man surrounded by a group of people is look...       sd   fake   

                           full_path  label  
0  ./data\real/hf_unsplash_41569.jpg      0  
1  ./data\real/hf_unsplash_28579.jpg      0  
2  ./data\real/hf_real_image_261.jpg  

### Step 3: Create a Custom PyTorch Dataset

This class is essential for loading data efficiently. For each item, it:
1. Reads an image from its file path.
2. Converts it to RGB format to ensure consistency.
3. Uses the SwinV2 `processor` to resize, normalize, and convert the image into a PyTorch tensor.

In [3]:
# [REPLACE THE ORIGINAL STEP 3 CELL WITH THIS ONE]

import torch
from torch.utils.data import Dataset
from PIL import Image
from transformers import AutoImageProcessor
from torchvision import transforms # Import the transforms module

# --- Model and Processor Setup ---
MODEL_CHECKPOINT = "microsoft/swinv2-small-patch4-window8-256"
processor = AutoImageProcessor.from_pretrained(MODEL_CHECKPOINT)

# --- Data Augmentation Pipeline ---
# Define a set of transformations to apply to the training images.
# These operations are applied randomly to each image as it's loaded.
train_transforms = transforms.Compose(
    [
        # Resize the image and crop it randomly to the model's input size.
        # This helps the model focus on different parts of the image.
        transforms.RandomResizedCrop(processor.size["height"]),

        # Flip the image horizontally with a 50% probability.
        transforms.RandomHorizontalFlip(p=0.5),

        # Slightly change the brightness, contrast, and saturation.
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),

        # Rotate the image by a small random angle.
        transforms.RandomRotation(10),
    ]
)

# --- Updated Custom Dataset Class ---
class ImageClassificationDataset(Dataset):
    # Add a 'transforms' argument to the constructor
    def __init__(self, df, processor, transforms=None):
        self.df = df
        self.processor = processor
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['full_path']
        label = self.df.iloc[idx]['label']

        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"Warning: File not found at {img_path}. Returning next item.")
            return self.__getitem__((idx + 1) % len(self))

        # ⭐️ Apply augmentations here, IF they are provided (i.e., for the training set)
        if self.transforms:
            image = self.transforms(image)

        # After optional augmentation, apply the model's required preprocessing
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)

        return {
            "pixel_values": pixel_values,
            "labels": torch.tensor(label, dtype=torch.long)
        }

# --- Create the Dataset Objects ---
# Apply the 'train_transforms' pipeline ONLY to the training dataset.
train_dataset = ImageClassificationDataset(train_df, processor, transforms=train_transforms)

# The validation dataset should NOT be augmented, so we pass transforms=None.
val_dataset = ImageClassificationDataset(val_df, processor, transforms=None)

print("✅ Dataset objects created successfully.")
print(f"Data augmentation pipeline is active for the training dataset.")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Dataset objects created successfully.
Data augmentation pipeline is active for the training dataset.


In [4]:
# import torch
# from torch.utils.data import Dataset
# from PIL import Image
# from transformers import AutoImageProcessor

# # --- Model and Processor Setup ---
# MODEL_CHECKPOINT = "microsoft/swinv2-small-patch4-window8-256"
# processor = AutoImageProcessor.from_pretrained(MODEL_CHECKPOINT)

# # --- Custom Dataset Class ---
# class ImageClassificationDataset(Dataset):
#     def __init__(self, df, processor):
#         self.df = df
#         self.processor = processor

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         # Get image path and label from the dataframe
#         img_path = self.df.iloc[idx]['full_path']
#         label = self.df.iloc[idx]['label']

#         # Open image, convert to RGB (handles grayscale, etc.)
#         try:
#             image = Image.open(img_path).convert("RGB")
#         except FileNotFoundError:
#             print(f"Warning: File not found at {img_path}. Returning next item.")
#             # A simple way to handle missing files is to load the next one
#             return self.__getitem__((idx + 1) % len(self))

#         # Apply transformations (resizing, normalization)
#         # return_tensors="pt" gives us PyTorch tensors
#         inputs = self.processor(images=image, return_tensors="pt")

#         # The processor output is batched, so we squeeze it to remove the batch dimension
#         pixel_values = inputs['pixel_values'].squeeze(0)

#         return {
#             "pixel_values": pixel_values,
#             "labels": torch.tensor(label, dtype=torch.long)
#         }

# # Create the dataset objects for training and validation
# train_dataset = ImageClassificationDataset(train_df, processor)
# val_dataset = ImageClassificationDataset(val_df, processor)

# print("\nDataset objects created successfully.")

### Step 4: Configure the Model and Training Arguments

We load the pre-trained SwinV2 model and modify its classification head for our binary task (`num_labels=2`). Then, we define the training hyperparameters using `TrainingArguments`.

In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

# --- Load the Model ---
# We specify num_labels=2 for our binary task (real/fake)
# ignore_mismatched_sizes=True allows us to replace the pre-trained classification head
from transformers import AutoModelForImageClassification

MODEL_CHECKPOINT = "microsoft/swinv2-small-patch4-window8-256"

# This line automatically finds and uses the correct class (SwinV2ForImageClassification)
model = AutoModelForImageClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=2,
    ignore_mismatched_sizes=True
)
# --- Evaluation Metric ---
# We'll use accuracy to monitor performance during training
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# --- Training Arguments ---
# This object contains all the hyperparameters for training
training_args = TrainingArguments(
    output_dir="./swinv2-real-vs-fake1",
    per_device_train_batch_size=8,  # Lower if you run out of GPU memory (e.g., 8)
    per_device_eval_batch_size=16,
    eval_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save a checkpoint at the end of each epoch
    num_train_epochs=25,              # Start with a few epochs; you can increase this later
    fp16=True,                       # Use mixed-precision training for speed (requires a T4/V100/A100 GPU)
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,               # Set to True if you want to upload to Hugging Face Hub
    remove_unused_columns=False,     # Important for our custom dataset
)

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

### Step 5: Start Training!

With everything set up, we instantiate the `Trainer` and call the `.train()` method. This will kick off the fine-tuning process. Colab will show a progress bar with the loss and, at the end of each epoch, the validation accuracy.

In [11]:
from transformers import EarlyStoppingCallback


# --- Instantiate the Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,             # The processor is passed here for data collating
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Add early stopping
)

# --- Start Training! ---
print("🚀 Starting training...")
trainer.train()  # Change to a checkpoint path to resume training
print("✅ Training finished!")

C:\Users\komal\AppData\Local\Temp\ipykernel_33672\3606930544.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.235600,0.288476,0.889442
2,0.218200,0.228412,0.931034
3,0.129400,0.350319,0.916104
4,0.126200,0.178832,0.956274
5,0.115700,0.201478,0.951653
6,0.123400,0.315777,0.939211


✅ Training finished!


In [7]:
trainer.save_model("./best-model")  # Save best model to a clean directory


### Step 6: Inference with the Fine-Tuned Model

After training, the best model is saved. Let's load it and use it to predict whether a new, unseen image is real or fake.

In [8]:
from PIL import Image

# The trainer saves the best model in the output directory specified in TrainingArguments
# The `load_best_model_at_end=True` argument ensures this is the one with the highest accuracy
best_model_path = trainer.state.best_model_checkpoint
print(f"Loading best model from: {best_model_path}")

# Load the fine-tuned model and processor
inference_model = AutoModelForImageClassification.from_pretrained(best_model_path)
inference_processor = AutoImageProcessor.from_pretrained(best_model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model.to(device)

# --- Make a prediction ---
# ⬇️ IMPORTANT: Change this path to an image you want to test
image_path = "data/fake_SD/00c461e5-9c5b-498c-937f-41bfc338565f.jpg" # Example path

try:
    image = Image.open(image_path).convert("RGB")

    # Process the image and move it to the GPU
    inputs = inference_processor(images=image, return_tensors="pt").to(device)

    # Get model outputs (logits)
    with torch.no_grad():
        outputs = inference_model(**inputs)

    # Get the predicted class
    predicted_class_idx = outputs.logits.argmax(-1).item()
    labels_map = ['real', 'fake'] # Corresponds to 0 and 1
    print(f"\nPrediction for '{os.path.basename(image_path)}':")
    print(f"--> Predicted class: {labels_map[predicted_class_idx]}")

except FileNotFoundError:
    print(f"\nError: The image file was not found at '{image_path}'.")
    print("Please update the 'image_path' variable to a valid file path in your Google Drive.")


Loading best model from: ./swinv2-real-vs-fake\checkpoint-8448

Prediction for '00c461e5-9c5b-498c-937f-41bfc338565f.jpg':
--> Predicted class: fake
